In [2]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = None #'password'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

Connected to Redis


True

# Predictive Analytics for New Product Lines

## Description of the use case
This use case involves using predictive analytics to forecast demand for new product lines. By analyzing historical sales data, businesses can make data-driven decisions about which products to launch and in what quantities.

## Redis Data Structures Used
- **TimeSeries**: To store and analyze time-series data of product sales.
- **Hashes**: To store product information.


In [3]:
r.flushdb()
# Mock data for the use case
mock_data = {
    'product_1001': {'name': 'Short Term FDs', 'category': 'savings'},
    'product_1002': {'name': 'Home Loan', 'category': 'loan'},
    'product_1003': {'name': 'Personal Loan', 'category': 'loan'},
}

# Use Hashes to store product information
for product_id, product_info in mock_data.items():
    r.hset(product_id, mapping=product_info)

# Use TimeSeries to store historical sales data
sales_data = {
    'product_1001': [(1622505600000, 100), (1625097600000, 150), (1627776000000, 200)],
    'product_1002': [(1622505600000, 80), (1625097600000, 120), (1627776000000, 160)],
    'product_1003': [(1622505600000, 60), (1625097600000, 90), (1627776000000, 130)],
}

for product_id, data_points in sales_data.items():
    for timestamp, value in data_points:
        r.ts().add(f'sales:{product_id}', timestamp, value)

# Function to forecast sales for the next period (example using simple moving average)
def forecast_sales(product_id, window=2):
    sales = r.ts().range(f'sales:{product_id}', from_time='-', to_time='+')
    if len(sales) < window:
        return None
    total = sum(value for timestamp, value in sales[-window:])
    return total // window

# Forecast sales for each product
for product_id in mock_data.keys():
    forecast = forecast_sales(product_id)
    print(f'Forecasted sales for {product_id}: {forecast} [Moving average of last 2 data points]')

# Print the sales data for each product
for product_id in mock_data.keys():
    sales = r.ts().range(f'sales:{product_id}', from_time='-', to_time='+')
    print(f'Sales data for {product_id}: {sales}')


Forecasted sales for product_1001: 175.0 [Moving average of last 2 data points]
Forecasted sales for product_1002: 140.0 [Moving average of last 2 data points]
Forecasted sales for product_1003: 110.0 [Moving average of last 2 data points]
Sales data for product_1001: [(1622505600000, 100.0), (1625097600000, 150.0), (1627776000000, 200.0)]
Sales data for product_1002: [(1622505600000, 80.0), (1625097600000, 120.0), (1627776000000, 160.0)]
Sales data for product_1003: [(1622505600000, 60.0), (1625097600000, 90.0), (1627776000000, 130.0)]
